Simple classifier based on (binary) HDC with [this dataset](https://www.kaggle.com/datasets/uciml/zoo-animal-classification).
First define operations

In [2]:
using Random

bitHDV(N::Int=10000) = bitrand(N)

function bitadd(vectors::BitVector ...)
    v = reduce(.+, vectors)
    n = length(vectors) / 2
    x = [i > n ? 1 : i < n ? 0 : rand(0:1) for i in v]
    return convert(BitVector, x)
end

bitbind(vectors::BitVector ...) =  reduce(.⊻, vectors)

bitperm(vector::BitVector, k::Int=1) = circshift(vector, k)

hamming(x::BitVector, y::BitVector) = sum(x .!= y)/length(x)

hamming (generic function with 1 method)

In [3]:
using DataFrames, CSV

In [4]:
data = CSV.read("zoo.csv", DataFrame)

,animal_name,hair,feathers,eggs,milk,airborne,aquatic,predator,toothed
,String15,Int64,Int64,Int64,Int64,Int64,Int64,Int64,Int64
1,aardvark,1,0,0,1,0,0,1,1
2,antelope,1,0,0,1,0,0,0,1
3,bass,0,0,1,0,0,1,1,1
4,bear,1,0,0,1,0,0,1,1
5,boar,1,0,0,1,0,0,1,1
6,buffalo,1,0,0,1,0,0,0,1
7,calf,1,0,0,1,0,0,0,1
8,carp,0,0,1,0,0,1,0,1
9,catfish,0,0,1,0,0,1,1,1


In [5]:
class_data = CSV.read("class.csv", DataFrame)

,Class_Number,Number_Of_Animal_Species_In_Class,Class_Type,Animal_Names
,Int64,Int64,String15,String
1,1,41,Mammal,"aardvark, antelope, bear, boar, buffalo, calf, cavy, cheetah, deer, dolphin, elephant, fruitbat, giraffe, girl, goat, gorilla, hamster, hare, leopard, lion, lynx, mink, mole, mongoose, opossum, oryx, platypus, polecat, pony, porpoise, puma, pussycat, raccoon, reindeer, seal, sealion, squirrel, vampire, vole, wallaby, wolf"
2,2,20,Bird,"chicken, crow, dove, duck, flamingo, gull, hawk, kiwi, lark, ostrich, parakeet, penguin, pheasant, rhea, skimmer, skua, sparrow, swan, vulture, wren"
3,3,5,Reptile,"pitviper, seasnake, slowworm, tortoise, tuatara"
4,4,13,Fish,"bass, carp, catfish, chub, dogfish, haddock, herring, pike, piranha, seahorse, sole, stingray, tuna"
5,5,4,Amphibian,"frog, frog, newt, toad"
6,6,8,Bug,"flea, gnat, honeybee, housefly, ladybird, moth, termite, wasp"
7,7,10,Invertebrate,"clam, crab, crayfish, lobster, octopus, scorpion, seawasp, slug, starfish, worm"


Make a HDV for every feature

In [6]:
feature_hdv=[bitHDV() for i in 1:17]

17-element Vector{BitVector}:
 [0, 1, 1, 0, 1, 0, 1, 0, 0, 1  …  0, 1, 0, 1, 0, 1, 0, 1, 1, 0]
 [1, 1, 0, 0, 0, 0, 1, 1, 1, 1  …  1, 0, 1, 1, 0, 1, 0, 1, 1, 1]
 [1, 0, 0, 1, 0, 0, 1, 0, 1, 0  …  0, 1, 1, 1, 0, 0, 1, 0, 0, 0]
 [1, 0, 0, 0, 1, 1, 0, 1, 0, 0  …  1, 1, 1, 0, 1, 1, 1, 1, 0, 0]
 [1, 1, 1, 0, 1, 0, 1, 1, 1, 1  …  0, 1, 0, 1, 1, 0, 0, 0, 1, 1]
 [1, 0, 0, 0, 0, 1, 0, 0, 1, 1  …  1, 1, 1, 1, 0, 1, 0, 1, 1, 1]
 [1, 0, 1, 0, 1, 0, 1, 1, 1, 0  …  0, 0, 0, 1, 1, 0, 0, 0, 1, 1]
 [0, 0, 1, 0, 1, 1, 0, 0, 0, 0  …  1, 1, 0, 0, 1, 1, 1, 0, 1, 0]
 [1, 0, 0, 0, 1, 1, 0, 0, 1, 0  …  1, 1, 1, 0, 1, 0, 1, 1, 1, 1]
 [0, 0, 0, 1, 0, 0, 1, 1, 1, 0  …  0, 1, 1, 1, 1, 1, 0, 1, 0, 0]
 [0, 1, 0, 1, 0, 0, 1, 1, 0, 1  …  0, 1, 0, 1, 1, 1, 0, 0, 1, 0]
 [1, 1, 0, 0, 0, 1, 0, 0, 1, 1  …  0, 1, 0, 0, 1, 1, 0, 1, 1, 0]
 [1, 0, 1, 1, 1, 0, 0, 0, 1, 1  …  1, 1, 0, 1, 1, 0, 1, 0, 0, 1]
 [0, 0, 1, 0, 1, 0, 1, 0, 1, 1  …  1, 0, 1, 0, 1, 1, 0, 1, 0, 0]
 [1, 0, 0, 1, 1, 1, 0, 1, 0, 1  …  0, 0, 1, 0, 1, 1, 0, 1, 0

In [7]:
Mammal = split(class_data.Animal_Names[1], ", ")
Bird = split(class_data.Animal_Names[2], ", ")
Reptile = split(class_data.Animal_Names[3], ", ")
Fish = split(class_data.Animal_Names[4], ", ")
Amphibian = split(class_data.Animal_Names[5], ", ")
Bug = split(class_data.Animal_Names[6], ", ")
Invertebrate = split(class_data.Animal_Names[7], ", ")

10-element Vector{SubString{String}}:
 "clam"
 "crab"
 "crayfish"
 "lobster"
 "octopus"
 "scorpion"
 "seawasp"
 "slug"
 "starfish"
 "worm"

Bundle for every species all of its containing feature HDVs to create a new HDV representing the species. 

In [8]:
hdv = BitVector[]
for i in 1:101
    v = BitVector[]
    for j in 2:18
        if data[!, j][i] == 1
            push!(v, feature_hdv[j-1])
        end
    end
    x = bitadd(v...)
    push!(hdv, x)
end

data[!, :species_hdv] = hdv


101-element Vector{BitVector}:
 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0  …  1, 1, 0, 1, 1, 1, 0, 1, 1, 0]
 [0, 0, 1, 0, 1, 0, 0, 0, 0, 0  …  1, 1, 0, 0, 1, 1, 1, 1, 0, 0]
 [1, 0, 0, 0, 1, 1, 0, 0, 1, 0  …  1, 1, 1, 0, 1, 1, 0, 1, 1, 0]
 [1, 0, 1, 0, 1, 0, 1, 1, 0, 0  …  0, 1, 0, 0, 1, 1, 0, 1, 1, 0]
 [1, 0, 1, 0, 1, 0, 1, 0, 0, 0  …  1, 1, 0, 0, 1, 1, 0, 1, 0, 0]
 [0, 0, 1, 0, 1, 0, 1, 0, 0, 0  …  1, 1, 1, 0, 1, 1, 0, 1, 0, 0]
 [1, 0, 0, 0, 1, 0, 0, 0, 0, 0  …  1, 1, 1, 0, 1, 1, 0, 1, 0, 0]
 [1, 0, 0, 0, 1, 1, 0, 0, 1, 1  …  1, 1, 1, 0, 1, 1, 0, 1, 1, 0]
 [1, 0, 0, 0, 1, 1, 0, 0, 1, 0  …  1, 1, 1, 0, 1, 1, 0, 1, 1, 0]
 [1, 0, 0, 0, 1, 1, 0, 0, 0, 0  …  1, 1, 1, 0, 1, 1, 1, 1, 0, 0]
 [1, 0, 1, 0, 1, 0, 1, 0, 0, 0  …  1, 1, 0, 0, 1, 1, 0, 1, 0, 0]
 [1, 0, 0, 0, 1, 0, 1, 1, 1, 1  …  0, 1, 1, 1, 1, 1, 0, 1, 0, 0]
 [1, 0, 0, 0, 1, 1, 0, 0, 1, 0  …  1, 1, 1, 0, 1, 1, 0, 1, 1, 0]
 ⋮
 [1, 0, 0, 0, 0, 1, 0, 0, 1, 0  …  1, 1, 1, 1, 1, 1, 1, 1, 1, 0]
 [1, 0, 0, 1, 0, 0, 1, 0, 1, 0  …  0, 1, 1, 0, 1, 1, 0, 

Let's bundle all animals of a group into a HDV representing the group.

In [13]:
function grouper(group::String, group2::Vector)
    v = BitVector[]
    for i in 1:101
        if data.animal_name[i] in group2
            push!(v, data.species_hdv[i])
        end
    end
        x = bitadd(v...)
        return x
    end

grouper (generic function with 1 method)

In [15]:
mammal_hdv = grouper("Mammal", Mammal)
bird_hdv = grouper("Bird", Bird)
reptile_hdv = grouper("Reptile", Reptile)
fish_hdv = grouper("Fish", Fish)
amphibian_hdv = grouper("Amphibian", Amphibian)
bug_hdv = grouper("Bug", Bug)
invertebrate_hdv = grouper("Invertebrate", Invertebrate)

10000-element BitVector:
 1
 0
 0
 0
 0
 0
 1
 0
 1
 0
 0
 0
 1
 ⋮
 0
 1
 0
 1
 1
 1
 0
 0
 0
 0
 1
 1

Let's compare the HDV of the flamingo to the HDVs of every animal group. 

In [20]:
println("mammal ",string(hamming(data.species_hdv[24], mammal_hdv)))
println("bird ",string(hamming(data.species_hdv[24], bird_hdv)))
println("reptile ",string(hamming(data.species_hdv[24], reptile_hdv)))
println("amphibian ",string(hamming(data.species_hdv[24], amphibian_hdv)))
println("bug ",string(hamming(data.species_hdv[24], bug_hdv)))
println("fish ",string(hamming(data.species_hdv[24], fish_hdv)))
println("invertebrate ",string(hamming(data.species_hdv[24], invertebrate_hdv)))

mammal 0.3203
bird 0.1042
reptile 0.2693
amphibian 0.313
bug 0.2852
fish 0.3428
invertebrate 0.4038


Like expected, the flamingo shows most similarity to the bird group! Let's try the animal vampire since I'm not sure what's ment by this...

In [21]:
println("mammal ",string(hamming(data.species_hdv[94], mammal_hdv)))
println("bird ",string(hamming(data.species_hdv[94], bird_hdv)))
println("reptile ",string(hamming(data.species_hdv[94], reptile_hdv)))
println("amphibian ",string(hamming(data.species_hdv[94], amphibian_hdv)))
println("bug ",string(hamming(data.species_hdv[94], bug_hdv)))
println("fish ",string(hamming(data.species_hdv[94], fish_hdv)))
println("invertebrate ",string(hamming(data.species_hdv[94], invertebrate_hdv)))

mammal 0.1917
bird 0.324
reptile 0.3107
amphibian 0.3332
bug 0.3618
fish 0.36
invertebrate 0.4876
